In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import adaptive

adaptive.notebook_extension()
hv.notebook_extension('matplotlib')

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

import kwant
import matplotlib.pyplot as plt
import supercurrent_matsubara
from functools import partial

In [ ]:
syst_pars = {
    'L_down':  100,
     'L_up': 100,
     'L_m': 40,
     'L_x': 1000,
     'a': 10,
     'mu_from_bottom_of_spin_orbit_bands': True,
     'with_vlead': True}

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 1.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = 0.18,
                 Delta_right = 0.18,
                 B = 0.5,
                 phase = 0,
                 T =.1,
                 V=0)

params = dict(**constants,
              **params_raw)
_=plotting_results.plot_syst(syst_pars, params)

In [ ]:
%%time
syst, hopping = sns_system.make_sns_system(**syst_pars)
kwant.plot(syst);
el_blocks = partial(sns_system.take_electron_blocks, norbs=4)
I_c = supercurrent_matsubara.I_c(syst, hopping, params, el_blocks)

In [ ]:
%%time 
syst_pars['L_x'] = 10
syst = sns_system.make_wrapped_system(**syst_pars)
kwant.plot(syst);

import supercurrent
def cur(phase):
    params['phase'] = phase
    I_c2 = supercurrent.wrapped_current(syst_pars, params, syst_wrapped=syst, tol=1e-3)
    return 1e9 * I_c2
phases = np.linspace(-3, 3, 31)
curs = np.array([cur(p) for p in phases])

In [ ]:
hv.Curve((phases, curs/curs.max())) * hv.Scatter((I_c['phases'], I_c['currents'] / I_c['currents'].max()))

In [ ]:
hv.Curve((phases, curs/4)) * hv.Scatter((I_c['phases'], I_c['currents'] / syst_pars['L_x']))

In [ ]:
hv.Curve((phases, curs/curs.max())) * hv.Scatter((I_c['phases'], I_c['currents'] / I_c['currents'].max()))

In [ ]:
curs = np.array(curs) - curs[15]
curs = curs/ curs.max()

In [ ]:
hv.Curve(curs[:15]) * hv.Curve(-curs[15:][::-1])

In [ ]:
c = I_c['currents'] / I_c['currents'].max()
hv.Curve(c[:15]) * hv.Curve(-c[15:][::-1])